In [ ]:
### Dit is oude, niet-werkende code die geen onderdeel vormt van de pipeline.
### This is old code that is not-working and not a part of the pipeline.

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM

model.tie_weights()

config = AutoConfig.from_pretrained("Mirage-Studio/llama-gaan-2-7b-chat-hf-dutch")
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

device_map = infer_auto_device_map(model)

In [16]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype, load_in_8bit_fp32_cpu_offload):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
        load_in_8bit_fp32_cpu_offload = load_in_8bit_fp32_cpu_offload,
    )

    return bnb_config

In [15]:
def load_model(model_name, bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """

    # Get number of GPU device and set maximum memory
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    # Load model tokenizer with the user authentication token
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)

    # Set padding token as EOS token
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [14]:
model_name = 'openlm-research/open_llama_3b'

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = False

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.bfloat16

load_in_8bit_fp32_cpu_offload = True

In [ ]:
# Load model from Hugging Face Hub with model name and bitsandbytes configuration

bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype, load_in_8bit_fp32_cpu_offload)

model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", offload_folder="offload", offload_state_dict = True, torch_dtype=torch.float16
)

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "robinsmits/open_llama_7b_alpaca_clean_dutch_qlora"

tokenizer =  AutoTokenizer.from_pretrained(model_name, use_fast = False, add_eos_token = True)

config = PeftConfig.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_8bit = True, device_map = "auto")
model = PeftModel.from_pretrained(model, model_name)

prompt = "### Instructie:\nWat zijn de drie belangrijkste softwareonderdelen die worden gebruikt bij webontwikkeling?\n\n### Antwoord:\n" 

inputs = tokenizer(prompt, return_tensors = "pt", truncation = True).input_ids.cuda()
sample = model.generate(input_ids = inputs, max_new_tokens = 512, num_beams = 2, early_stopping = True, eos_token_id = tokenizer.eos_token_id)
output = tokenizer.decode(sample[0], skip_special_tokens = True)

print(output.split(prompt)[1])

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│ /data/milenadeswart/.local/lib/python3.7/site-packages/transformers/models/auto/auto_factory.py: │
│ 485 in from_pretrained                                                                           │
│                                                                                                  │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   483 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│   484 │   │   │   return model_class.from_pretrained(                                            │
│ ❱ 485 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   486 │   │   │   )                                                                              │
│   487 │   │   raise ValueError(                                                                  │
│   488 │   │   │   f"Unrecognized configuration class {config.__class__} for this kind of AutoM   │
│                                                                                                  │
│ /data/milenadeswart/.local/lib/python3.7/site-packages/transformers/modeling_utils.py:2827 in    │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   2824 │   │   │   │   │   │   `device_map` to `from_pretrained`. Check                          │
│   2825 │   │   │   │   │   │   https://huggingface.co/docs/transformers/main/en/main_classes/qu  │
│   2826 │   │   │   │   │   │   for more details.                                                 │
│ ❱ 2827 │   │   │   │   │   │   """                                                               │
│   2828 │   │   │   │   │   )                                                                     │
│   2829 │   │   │   │   del device_map_without_lm_head                                            │
│   2830                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to 
fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a 
custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-
cpu-and-gpu
                        for more details.

In [6]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

model_path = 'robinsmits/open_llama_7b_alpaca_clean_dutch_qlora'
# model_path = 'openlm-research/open_llama_7b'

tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

prompt = 'Q: What is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=32
)
print(tokenizer.decode(generation_output[0]))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /data/milenadeswart/.local/lib/python3.7/site-packages/huggingface_hub/utils/_errors.py:261 in   │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   258 │   </Tip>                                                                                 │
│   259 │   """                                                                                    │
│   260 │   try:                                                                                   │
│ ❱ 261 │   │   response.raise_for_status()                                                        │
│   262 │   except HTTPError as e:                                                                 │
│   263 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   264                                                                                            │
│                                                                                                  │
│ /data/anaconda3/lib/python3.7/site-packages/requests/models.py:940 in raise_for_status           │
│                                                                                                  │
│   937 │   │   │   http_error_msg = u'%s Server Error: %s for url: %s' % (self.status_code, rea   │
│   938 │   │                                                                                      │
│   939 │   │   if http_error_msg:                                                                 │
│ ❱ 940 │   │   │   raise HTTPError(http_error_msg, response=self)                                 │
│   941 │                                                                                          │
│   942 │   def close(self):                                                                       │
│   943 │   │   """Releases the connection back to the pool. Once this method has been             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: 
https://huggingface.co/robinsmits/open_llama_7b_alpaca_clean_dutch_qlora/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /data/milenadeswart/.local/lib/python3.7/site-packages/transformers/utils/hub.py:429 in          │
│ cached_file                                                                                      │
│                                                                                                  │
│    426 │   │   │   proxies=proxies,                                                              │
│    427 │   │   │   resume_download=resume_download,                                              │
│    428 │   │   │   use_auth_token=use_auth_token,                                                │
│ ❱  429 │   │   │   local_files_only=local_files_only,                                            │
│    430 │   │   )                                                                                 │
│    431 │                                                                                         │
│    432 │   except RepositoryNotFoundError:                                                       │
│                                                                                                  │
│ /data/milenadeswart/.local/lib/python3.7/site-packages/huggingface_hub/utils/_validators.py:118  │
│ in _inner_fn                                                                                     │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:     